<a href="https://colab.research.google.com/github/infinite-darkness108/EEG_analysis/blob/main/MNE_DATA_VISUALIZATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NOT FOR COLAB ; COPY IT TO YOUR PYTHON IDLE


MUST BE PIP INSTALLED

1.   pip install mne
2.   pip install numpy
3.   pip install pyvista
4.   pip install pyvistaqt
5.   pip install IPython
6.   pip install ipywidgets
7.   pip install ipyevents

By the way, No. 3 to 7 are only used for 3D PLOTS in IDLE


In [ ]:
import mne
import numpy as np

# How to visualize the sensors in both 2d top view and 3d. 
# clicking the dots reveal what exact sensor was placed at that particular scalp.

raw_path = r'C:\Users\91892\Desktop\MNE_EEG_SPEECH\MNE-sample-data\MEG\sample\sample_audvis_raw.fif'
raw = mne.io.read_raw_fif(raw_path)
raw.plot_sensors(kind='topomap', ch_type='mag')
raw.plot_sensors(kind='3d', ch_type='mag')

#_______________________________________________________________________________________________________________________



#In MNE, Evoked data is the average of multiple trials
#        that have been time-locked to a specifiC stimulus

#This can be useful for understanding the timing and location
#         of brain activity in response to different stimuli


evk_file = r'C:\Users\91892\Desktop\MNE_EEG_SPEECH\MNE-sample-data\MEG\sample\sample_audvis-ave.fif'
evokeds_list = mne.read_evokeds(evk_file, baseline=(None, 0), proj=True, verbose=False)

# Show condition names and baseline intervals
for e in evokeds_list:
    print(f'Condition: {e.comment}, baseline: {e.baseline}')


# HOW TO INTERPRET THE OUTPUT?


# FOR EXAMPLE "Condition: Left Auditory,
#              baseline: (-0.19979521315838786, 0.0)"
# IS INTERPRETED AS FOLLOWS

# A SAMPLE SOUND WAS PLAYED TO THE PARTICIPANT'S LEFT EAR.
# THEN THE CORRESPONDING EEG DATA WAS RECORDED.
# BASELINE IS THE TIME PERIOD BEFORE A STIMULUS OF INTEREST.
# BASELINE ACTIVITY IS CONSIDERED AS NOISE THAT MUST BE REMOVED.
# THAT NOISE CAN BE DUE TO BLINKING OF EYES, ELECTRICAL ACTIVITIES OF MUSCLES, ENVIRONMENTAL NOISE

# 0.19979521315838786 BEFORE THE START, BASELINE NOISE WAS PRESENT .

# USUALLY IN SUCH STUDIES THE PARTICIPANTS WILL BE GIVEN A TIMEFRAME WHERE THEY CAN BLINK (TO CONVENIENTLY DELETE)
# OTHER TIMES, THEY ARE ASKED TO ONLY RESPOND TO THE STIMULI 

#______________________________________________________________________________________________________________________

#use /-separated dictionary keys to encode the conditions (like is often done when epoching
#        because some of the plotting methods can take advantage of that style of coding.
#        epochs are segments of data that correspond to specific events or stimuli

conds = ('aud/left', 'aud/right', 'vis/left', 'vis/right')
evks = dict(zip(conds, evokeds_list))    


# BUTTERFLY PLOTS FOR EACH CHANNEL TYPES
# HERE EACH CHANNEL CORRESPONDS TO AN ELECTRODE ATTACHED TO A PARTICULAR POINT ON OUR HEAD (SCALP)
# WHEN YOU SUPERIMPOSE ALL THE SENSOR'S DATA IN TIMESERIES, THE PLOT YOU GET IS CALLED BUTTERFLY PLOT

evks['aud/left'].plot(exclude=[])

# FOR EXCLUDE PARAMETER, default is exclude='bads' WHICH WILL SUPPRESS THE BAD CHANNELS.
# FOR SOME REASON THE ELECTRODES HAVE MALFUNCTIONED (BAD CHANNEL), SO IT CAN REMOVED FROM ANALYSIS
# HERE IN THE ABOVE LINE OF CODE, WE HAVE NOT EXCLUDED ANY BAD CHANNEL, IF ANY EXISTED


# HOW TO INTERPRET THE OUTPUT?


# WE GET 4 PLOTS - EEG PLOT, GRADIOMETERS, MAGNETOMETERS, EOG

# EEG PLOT IS OBTAINED FROM THE ELECTRODES WHICH ARE NEEDLE LIKE AND ARE ATTACHED TO THE SCALP.
#          WHEN THE NEURONS FIRE, THEY CREATE SMALL ELECTRICAL CURRENT DETECTED BY THIS ELECTRODE.

# MAGNETOMETER PLOTS ARE FROM THE MAGNETOMETERS PLACED ON A SCALP.IT IS MADE OF MAGNET WITH METAL WIRES COILED ON IT.
#          WHEN THE EEG MAGNETIC WAVES ARE EMANATED OUT OF THE BRAIN, THESE VARYING MAGNETIC FIELDS WILL CREATE
#          CURRENT AND POTENTIAL DIFFERENCE ACROSS THE COIL (METALLIC WIRE). THIS VOLTAGE DATA IS THE MAGNETOMETER PLOT.

# GRADIOMETERS MEASURE THE SPATIAL GRADIENT OF THE MAGNETIC FIELD. THEY CONSIST OF 2 OR MORE MAGNETOMETERS PLACED CLOSELY.
#          BY SUBTRACTING THE MEASUREMENTS FROM EACH OTHER, GRADIENT OF THE MAGNETIC FIELD IS CALCULATED.

# EOG PLOT IS THE PLOT OF SIGNALS RECORDED BY ELECTRODES PLACED NEAR THE EYES.


# PLOTS THE RMS FOR THE SAME WITH HIGHLIGHTING INTERESTING TIME PERIODS

time_ranges_of_interest = [
    (0.05, 0.14),
    (0.22, 0.27)
]
evks['aud/left'].plot(
    picks='mag', spatial_colors=True, gfp=True,
    highlight=time_ranges_of_interest
)
# gpf = True IMPLIES THAT WE SUPERIMPOSE THE RMS OF THE SIGNAL FOR EVERY CHANNEL

#______________________________________________________________________________________________________________________



times = np.linspace(0.05, 0.13, 5)
evks['aud/left'].plot_topomap(ch_type='mag', times=times, colorbar=True)

# HOW TO INTERPRET THE OUTPUT?

# FIRST TOPOGRAPHY IS AVERAGED FOR 0 to 0.05 seconds, SECOND ONE 0.05 TO 0.07 ETC.,.



#SCALP TOPOGRAPHIES FOR DIFFERENT TIME DURATIONS

times = np.linspace(0.05, 0.13, 5)

averaging_durations = [0.01, 0.02, 0.03, None, None]
fig = evks['aud/left'].plot_topomap(
    ch_type='mag', times=times, average=averaging_durations
)

# HOW TO INTERPRET THE OUTPUT?

# HERE [0.01, 0.02, 0.03] WINDOW LENGTH IN TIME FOR RESPECTIVE SCALP TOPOGRAPHIES
# NONE IMPLIES THAT NO TIME AVERAGING ARE APPLIED TO 4TH AND 5TH TIME POINTS



#_____________________________________________________________________________________________________________________

# ARROW MAPS TO ESTIMATE THE MAGNITUDE AND DIRECTION OF THE MAGNETIC FIELDS

mags = evks['aud/left'].copy().pick_types(meg='mag')
mne.viz.plot_arrowmap(mags.data[:, 175], mags.info, extrapolate='local')

#_______________________________________________________________________________

# JOINT PLOTS HAVE BOTH BUTTERFLY PLOTS AND SCALP TOPOGRAPHIES
# THE TOPOGRAPHIES FOR EACH CHANNEL CORRESPOND TO THE PEAKS IN THE PLOT
evks['vis/right'].plot_joint()
# IF NO PICKS ARE SPECIFIED, THEN FIGURES FOR EACH "CHANNEL TYPE" ARE GENERATED


# EXAMPLES FOR CHANNEL TYPES
    # EEG -> ELECTRICAL ACTIVITY OF BRAIN 
    # MEG -> MAGNETIC FIELDS FROM BRAIN
    # EOG -> MEASURE EYE MOVEMENTS
    # EMG -> MEASURE MUSCLE ACTIVITY


#_______________________________________________________________________________

# COMPARING EVOKED DATA FROM DIFFERENT EXPERIMENTAL CONDITIONS
# PLOT MEAN, MEDIAN, RMS, AND CALL A CUSTOM FUNCTION

def custom_func(x):
    return x.max(axis=1)


for combine in ('mean', 'median', 'gfp', custom_func):
    mne.viz.plot_compare_evokeds(evks, picks='eeg', combine=combine)

# HERE THE CUSTOM FUNCTION RETURNS THE MAXIUMUM VALUE OF EACH ROW
# HERE ONLY THE EEG CHANNELS ARE PICKED FOR DISPLAY

#_______________________________________________________________________________

# CUSTOMISE SOILD OR DASHED LINES IN THE PLOT

mne.viz.plot_compare_evokeds(evks, picks='MEG 1811', colors=dict(aud=0, vis=1),
                             linestyles=dict(left='solid', right='dashed'),
                             time_unit='ms')


#_____________________________________________________________________________________________________________________


#plot_image() shows one channel per row

evks['vis/right'].plot_image()

#HOW TO INTERPRET THE OUTPUT?

# THE COLOR REPRESENTS THE STRENGTH OF THE SIGNAL'S MAGNETIC FIELD
# X AXIS IS TIME
# Y AXIS IS CHANNEL INDEX


#______________________________________________________________________________________________________________________


# TOPOGRAPHICAL SUBPLOTS ARE PLOTS OF EVERY SENSOR USED IN 2D (TOP VIEW)
# ALONG WITH LITTLE GRAPHS TO LOOK INTO AMPLITUDE / POWER OF THE SIGNALS .
# CLICK THOSE GRAPHS TO ENLARGE THEM.
mne.viz.plot_compare_evokeds(evks, picks='eeg', colors=dict(aud=0, vis=1),
                             linestyles=dict(left='solid', right='dashed'),
                             axes='topo', styles=dict(aud=dict(linewidth=1),
                                                      vis=dict(linewidth=1)))

#______________________________________________________________________________________________________________________


import pyvista
import pyvistaqt
import IPython
import ipywidgets
import ipyevents

# 3D FIELD MAPS REQUIRE A "TRANS FILE" FROM WHICH THE COORDINATES OF MEG DEVICES AND INFO ON HEAD ARE OBTAINED

trans_file = r'C:\Users\91892\Desktop\MNE_EEG_SPEECH\MNE-sample-data\MEG\sample\sample_audvis_raw-trans.fif'
subjects_dir = r'C:\Users\91892\Desktop\MNE_EEG_SPEECH\MNE-sample-data\subjects'

maps = mne.make_field_map(evks['aud/left'], trans=str(trans_file),
                          subject='sample', subjects_dir=subjects_dir)
evks['aud/left'].plot_field(maps, time=0.1)


# HOW TO INTERPRET THE OUTPUT?

# HERE THE BLUE COLOR REPRESENTS NEGATIVE POLARITY OF THE MAGNETIC FIELD (NEGATIVE VOLTAGE AT THE MEG SENSOR)
#      THE RED COLOR REPRESENTS POSITIVE POLARITY OF THE MAGNETIC FIELD  (POSITIVE VOLTAGE AT THE MEG SENSOR)


for ch_type in ('mag', 'grad', 'eeg'):
    evk = evks['aud/right'].copy().pick(ch_type)
    _map = mne.make_field_map(evk, trans=str(trans_file), subject='sample',
                              subjects_dir=subjects_dir, meg_surf='head')
    fig = evk.plot_field(_map, time=0.1)
    mne.viz.set_3d_title(fig, ch_type, size=20)
#________________________________________________________________________________________________________________